In [4]:
import dart_fss as dart
import pandas as pd

In [16]:
df_list = pd.read_csv('data/c_list.csv')

In [17]:
df_list.head()

,name
0,데코앤이
1,신텍
2,금오하이텍
3,에이치앤아이
4,티에스엠텍


In [18]:
date_list = pd.read_csv('data/com_list.csv')

In [38]:
date_list

,company,date
0,데코앤이,20191120
1,신텍,20180709
2,금오하이텍,20170607
3,에이치앤아이,20160714
4,티에스엠텍,20150520
...,...,...
504,홍익엔지니어링(주),20190402
505,홍주실업(주),20181024
506,화신산업(주),20191126
507,효진종합건설(주),20181204


In [20]:
date_list['company'] = [cmp_lst.strip() for cmp_lst in date_list['company']]

In [21]:
df_list['date'] = 0
for i in range(len(df_list['name'])):
    for j, k in zip(date_list['company'], date_list['date']):
        if df_list['name'][i] == j:
            df_list['date'][i] = k

<ipython-input-21-7598d54e5756>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list['date'][i] = k


In [22]:
df_list.head()

,name,date
0,데코앤이,20191120
1,신텍,20180709
2,금오하이텍,20170607
3,에이치앤아이,20160714
4,티에스엠텍,20150520


In [23]:
df_list['bgn_de'] = [date - 100000 for date in df_list['date']]
df_list.rename(columns = {'date' : 'end_de'}, inplace = True)

In [5]:
api_key='d0e1a8c463dcec6977b8104ed02b3aca61173fb8'
dart.set_api_key(api_key)
crp_list = dart.get_corp_list()

Output()

Output()

Output()

In [26]:
df_list

,name,end_de,bgn_de
0,데코앤이,20191120,20091120
1,신텍,20180709,20080709
2,금오하이텍,20170607,20070607
3,에이치앤아이,20160714,20060714
4,티에스엠텍,20150520,20050520
5,우양에이치씨,20150317,20050317
6,배명금속,20120903,20020903
7,금강제강,20120814,20020814
8,풍림산업,20120518,20020518
9,미주제강,20120417,20020417


In [29]:
none_financial = []
none_cis = []
none_bs = []
none_cf = []

In [31]:
for cmp_list, bgn_de, end_de in zip(df_list['name'], df_list['bgn_de'], df_list['end_de']):
    basic_info = crp_list.find_by_corp_name(cmp_list, exactly=True)[0]
    
    try:
        print(cmp_list)
        fs = basic_info.extract_fs(bgn_de=bgn_de, end_de=end_de)
    
    except:
        print('연결된 재무제표를 찾을 수 없음')
        none_financial.append(cmp_list)
        continue
    
    try:
        df_cis = fs['cis']
        df_cis.to_csv('data/포괄손익계산서/' + cmp_list + ' 포괄손익계산서 원본.csv', index=False)
    except:
        print(cmp_list+'의 포괄손익계산서가 없습니다.')
        none_cis.append(cmp_list)
        pass
    
    try:
        df_bs = fs['bs']
        df_bs.to_csv('data/재무상태표/' + cmp_list + ' 재무상태표 원본.csv', index=False)
    except:
        print(cmp_list+'의 재무상태표가 없습니다.')
        none_bs.append(cmp_list)
        pass
    
    try:
        df_cf = fs['cf']
        df_cf.to_csv('data/현금흐름도/' + cmp_list + ' 현금흐름표 원본.csv', index=False)
    except:
        print(cmp_list+'의 현금흐름도가 없습니다.')
        none_cf.append(cmp_list)
        pass
    


데코앤이


Output()

Output()


신텍


Output()

...

연결된 재무제표를 찾을 수 없음
우영
연결된 재무제표를 찾을 수 없음
알루코
연결된 재무제표를 찾을 수 없음
케이티씨텔레콤
연결된 재무제표를 찾을 수 없음
아이텍스필
연결된 재무제표를 찾을 수 없음
현주컴퓨터
연결된 재무제표를 찾을 수 없음
넥스텔
연결된 재무제표를 찾을 수 없음
지에스엔텍
연결된 재무제표를 찾을 수 없음
현대멀티캡
연결된 재무제표를 찾을 수 없음
중앙제지
연결된 재무제표를 찾을 수 없음
테크메이트
연결된 재무제표를 찾을 수 없음
텔슨전자
연결된 재무제표를 찾을 수 없음
금강화섬
연결된 재무제표를 찾을 수 없음
호성
연결된 재무제표를 찾을 수 없음
대호
연결된 재무제표를 찾을 수 없음
모닷텔
연결된 재무제표를 찾을 수 없음
화림모드



화림모드의 포괄손익계산서가 없습니다.
화림모드의 현금흐름도가 없습니다.
서통



서통의 포괄손익계산서가 없습니다.
계몽사



연결된 재무제표를 찾을 수 없음
코리아링크
연결된 재무제표를 찾을 수 없음
시스컴
연결된 재무제표를 찾을 수 없음
정산애강
연결된 재무제표를 찾을 수 없음
소프트윈
연결된 재무제표를 찾을 수 없음
심스밸리
연결된 재무제표를 찾을 수 없음
코닉스
연결된 재무제표를 찾을 수 없음
유니씨앤티
연결된 재무제표를 찾을 수 없음
아이씨켐
연결된 재무제표를 찾을 수 없음
디지텔
연결된 재무제표를 찾을 수 없음
삼한콘트롤스
연결된 재무제표를 찾을 수 없음
피어리스



피어리스의 포괄손익계산서가 없습니다.
정일이엔씨
연결된 재무제표를 찾을 수 없음
삼산
연결된 재무제표를 찾을 수 없음
교하산업
연결된 재무제표를 찾을 수 없음
아진산업
연결된 재무제표를 찾을 수 없음
주화산업
연결된 재무제표를 찾을 수 없음
라인건설
연결된 재무제표를 찾을 수 없음
대륭산업
연결된 재무제표를 찾을 수 없음
한국종합철관
연결된 재무제표를 찾을 수 없음


In [35]:
'''
시작년도를 5년전으로 해도 재무제표가 없는 기업이 44업체
재무제표가 있는 기업이여도 포괄손익계산서 없는 업체가 총 5업체
현금 흐름도가 없는 업체는 총 1개 업체
제무상태표는 모두 있는 상태.
'''
len(none_financial), len(none_cis), len(none_bs), len(none_cf)

(44, 6, 0, 1)

In [40]:
# 연결된 재무제표가 없는 기업들
for nf in none_financial:
    print(nf)

에이치앤아이
티에스엠텍
우양에이치씨
제이콤
인네트
신세계톰보이
에스피코프
쌈지
유리이에스
아이비진
우영
알루코
케이티씨텔레콤
아이텍스필
현주컴퓨터
넥스텔
지에스엔텍
현대멀티캡
중앙제지
테크메이트
텔슨전자
금강화섬
호성
대호
모닷텔
계몽사
코리아링크
시스컴
정산애강
소프트윈
심스밸리
코닉스
유니씨앤티
아이씨켐
디지텔
삼한콘트롤스
정일이엔씨
삼산
교하산업
아진산업
주화산업
라인건설
대륭산업
한국종합철관


In [41]:
# 연결된 재무제표가있으나 포괄손익 계산서가 없는 업체
for nf in none_cis:
    print(nf)

금오하이텍
금오하이텍
카라반케이디이
화림모드
서통
피어리스


In [42]:
# 연결된 재무제표가있으나 제무상태표가 없는 업체
for nf in none_bs:
    print(nf)

In [43]:
# 연결된 재무제표가있으나 현금 흐름도가 없는 업체
for nf in none_cf:
    print(nf)

화림모드


In [36]:
start_none_financial = []
start_none_cis = []
start_none_bs = []
start_none_cf = []

In [37]:
for cmp_list, end_de in zip(df_list['name'], df_list['end_de']):
    basic_info = crp_list.find_by_corp_name(cmp_list, exactly=True)[0]
    
    try:
        print(cmp_list)
        fs = basic_info.extract_fs(bgn_de='19990101', end_de=end_de)
    
    except:
        print('연결된 재무제표를 찾을 수 없음')
        start_none_financial.append(cmp_list)
        continue
    
    try:
        df_cis = fs['cis']
        df_cis.to_csv('data/1999_포괄손익계산서/' + cmp_list + ' 포괄손익계산서 원본.csv', index=False)
    except:
        print(cmp_list+'의 포괄손익계산서가 없습니다.')
        start_none_cis.append(cmp_list)
        pass
    
    try:
        df_bs = fs['bs']
        df_bs.to_csv('data/1999_재무상태표/' + cmp_list + ' 재무상태표 원본.csv', index=False)
    except:
        print(cmp_list+'의 재무상태표가 없습니다.')
        start_none_bs.append(cmp_list)
        pass
    
    try:
        df_cf = fs['cf']
        df_cf.to_csv('data/1999_현금흐름도/' + cmp_list + ' 현금흐름표 원본.csv', index=False)
    except:
        print(cmp_list+'의 현금흐름도가 없습니다.')
        start_none_cf.append(cmp_list)
        pass
    


데코앤이


Output()

Output()


신텍


Output()

...

연결된 재무제표를 찾을 수 없음
우영
연결된 재무제표를 찾을 수 없음
알루코
연결된 재무제표를 찾을 수 없음
케이티씨텔레콤
연결된 재무제표를 찾을 수 없음
아이텍스필
연결된 재무제표를 찾을 수 없음
현주컴퓨터
연결된 재무제표를 찾을 수 없음
넥스텔
연결된 재무제표를 찾을 수 없음
지에스엔텍
연결된 재무제표를 찾을 수 없음
현대멀티캡
연결된 재무제표를 찾을 수 없음
중앙제지
연결된 재무제표를 찾을 수 없음
테크메이트
연결된 재무제표를 찾을 수 없음
텔슨전자
연결된 재무제표를 찾을 수 없음
금강화섬
연결된 재무제표를 찾을 수 없음
호성
연결된 재무제표를 찾을 수 없음
대호
연결된 재무제표를 찾을 수 없음
모닷텔
연결된 재무제표를 찾을 수 없음
화림모드



화림모드의 포괄손익계산서가 없습니다.
화림모드의 현금흐름도가 없습니다.
서통



서통의 포괄손익계산서가 없습니다.
계몽사



연결된 재무제표를 찾을 수 없음
코리아링크
연결된 재무제표를 찾을 수 없음
시스컴
연결된 재무제표를 찾을 수 없음
정산애강
연결된 재무제표를 찾을 수 없음
소프트윈
연결된 재무제표를 찾을 수 없음
심스밸리
연결된 재무제표를 찾을 수 없음
코닉스
연결된 재무제표를 찾을 수 없음
유니씨앤티
연결된 재무제표를 찾을 수 없음
아이씨켐
연결된 재무제표를 찾을 수 없음
디지텔
연결된 재무제표를 찾을 수 없음
삼한콘트롤스
연결된 재무제표를 찾을 수 없음
피어리스



피어리스의 포괄손익계산서가 없습니다.
정일이엔씨
연결된 재무제표를 찾을 수 없음
삼산
연결된 재무제표를 찾을 수 없음
교하산업
연결된 재무제표를 찾을 수 없음
아진산업
연결된 재무제표를 찾을 수 없음
주화산업
연결된 재무제표를 찾을 수 없음
라인건설
연결된 재무제표를 찾을 수 없음
대륭산업
연결된 재무제표를 찾을 수 없음
한국종합철관
연결된 재무제표를 찾을 수 없음


In [53]:
'''
시작년도를 5년전으로 해도 재무제표가 없는 기업이 44업체
재무제표가 있는 기업이여도 포괄손익계산서 없는 업체가 총 5개 업체
현금 흐름도가 없는 업체는 총 1개 업체
제무상태표는 모두 있는 상태.
'''
len(start_none_financial), len(start_none_cis), len(start_none_bs), len(start_none_cf)

(44, 5, 0, 1)

In [44]:
# 연결된 재무제표가 없는 기업들
for nf in start_none_financial:
    print(nf)

에이치앤아이
티에스엠텍
우양에이치씨
제이콤
인네트
신세계톰보이
에스피코프
쌈지
유리이에스
아이비진
우영
알루코
케이티씨텔레콤
아이텍스필
현주컴퓨터
넥스텔
지에스엔텍
현대멀티캡
중앙제지
테크메이트
텔슨전자
금강화섬
호성
대호
모닷텔
계몽사
코리아링크
시스컴
정산애강
소프트윈
심스밸리
코닉스
유니씨앤티
아이씨켐
디지텔
삼한콘트롤스
정일이엔씨
삼산
교하산업
아진산업
주화산업
라인건설
대륭산업
한국종합철관


In [45]:
# 연결된 재무제표가있으나 포괄손익 계산서가 없는 업체
for nf in start_none_cis:
    print(nf)

금오하이텍
카라반케이디이
화림모드
서통
피어리스


In [46]:
# 연결된 재무제표가있으나 제무상태표가 없는 업체
for nf in start_none_bs:
    print(nf)

In [47]:
# 연결된 재무제표가있으나 현금 흐름도가 없는 업체
for nf in start_none_cf:
    print(nf)

화림모드


In [54]:
a = 0
for nf in none_financial:
    for start in start_none_financial:
        if nf == start:
            a += 1
        else:
            continue

print(a)
    

44


In [55]:
b = 0
for nf in none_cis:
    for start in start_none_cis:
        if nf == start:
            b += 1
        else:
            continue

print(b)

6


In [56]:
c = 0
for nf in none_bs:
    for start in start_none_bs:
        if nf == start:
            c += 1
        else:
            continue

print(c)

0


In [57]:
d = 0
for nf in none_cf:
    for start in start_none_cf:
        if nf == start:
            d += 1
        else:
            continue

print(d)

1


In [6]:
df_list = pd.read_csv('data/c_list.csv')

In [7]:
date_list = pd.read_csv('data/com_list.csv')

In [8]:
date_list['company'] = [cmp_lst.strip() for cmp_lst in date_list['company']]

In [9]:
df_list['date'] = 0
for i in range(len(df_list['name'])):
    for j, k in zip(date_list['company'], date_list['date']):
        if df_list['name'][i] == j:
            df_list['date'][i] = k

<ipython-input-9-7598d54e5756>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list['date'][i] = k


In [10]:
df_list['bgn_de'] = [date - 30000 for date in df_list['date']]
df_list.rename(columns = {'date' : 'end_de'}, inplace = True)

In [11]:
thrd_none_financial = []
thrd_none_cis = []
thrd_none_bs = []
thrd_none_cf = []

In [12]:
for cmp_list, bgn_de, end_de in zip(df_list['name'], df_list['bgn_de'], df_list['end_de']):
    basic_info = crp_list.find_by_corp_name(cmp_list, exactly=True)[0]
    
    try:
        print(cmp_list)
        fs = basic_info.extract_fs(bgn_de=bgn_de, end_de=end_de)
    
    except:
        print('연결된 재무제표를 찾을 수 없음')
        thrd_none_financial.append(cmp_list)
        continue
    
    try:
        df_cis = fs['cis']
        df_cis.to_csv('data/3rd_포괄손익계산서/' + cmp_list + ' 포괄손익계산서 원본.csv', index=False)
    except:
        print(cmp_list+'의 포괄손익계산서가 없습니다.')
        thrd_none_cis.append(cmp_list)
        pass
    
    try:
        df_bs = fs['bs']
        df_bs.to_csv('data/3rd_재무상태표/' + cmp_list + ' 재무상태표 원본.csv', index=False)
    except:
        print(cmp_list+'의 재무상태표가 없습니다.')
        thrd_none_bs.append(cmp_list)
        pass
    
    try:
        df_cf = fs['cf']
        df_cf.to_csv('data/3rd_현금흐름도/' + cmp_list + ' 현금흐름표 원본.csv', index=False)
    except:
        print(cmp_list+'의 현금흐름도가 없습니다.')
        thrd_none_cf.append(cmp_list)
        pass
    


데코앤이


Output()

Output()


신텍


Output()

...

연결된 재무제표를 찾을 수 없음
우영
연결된 재무제표를 찾을 수 없음
알루코
연결된 재무제표를 찾을 수 없음
케이티씨텔레콤
연결된 재무제표를 찾을 수 없음
아이텍스필
연결된 재무제표를 찾을 수 없음
현주컴퓨터
연결된 재무제표를 찾을 수 없음
넥스텔
연결된 재무제표를 찾을 수 없음
지에스엔텍
연결된 재무제표를 찾을 수 없음
현대멀티캡
연결된 재무제표를 찾을 수 없음
중앙제지
연결된 재무제표를 찾을 수 없음
테크메이트
연결된 재무제표를 찾을 수 없음
텔슨전자
연결된 재무제표를 찾을 수 없음
금강화섬
연결된 재무제표를 찾을 수 없음
호성
연결된 재무제표를 찾을 수 없음
대호
연결된 재무제표를 찾을 수 없음
모닷텔
연결된 재무제표를 찾을 수 없음
화림모드



화림모드의 포괄손익계산서가 없습니다.
화림모드의 현금흐름도가 없습니다.
서통



서통의 포괄손익계산서가 없습니다.
계몽사



계몽사의 포괄손익계산서가 없습니다.
코리아링크
연결된 재무제표를 찾을 수 없음
시스컴
연결된 재무제표를 찾을 수 없음
정산애강
연결된 재무제표를 찾을 수 없음
소프트윈
연결된 재무제표를 찾을 수 없음
심스밸리
연결된 재무제표를 찾을 수 없음
코닉스
연결된 재무제표를 찾을 수 없음
유니씨앤티
연결된 재무제표를 찾을 수 없음
아이씨켐
연결된 재무제표를 찾을 수 없음
디지텔
연결된 재무제표를 찾을 수 없음
삼한콘트롤스
연결된 재무제표를 찾을 수 없음
피어리스



피어리스의 포괄손익계산서가 없습니다.
정일이엔씨
연결된 재무제표를 찾을 수 없음
삼산
연결된 재무제표를 찾을 수 없음
교하산업
연결된 재무제표를 찾을 수 없음
아진산업
연결된 재무제표를 찾을 수 없음
주화산업
연결된 재무제표를 찾을 수 없음
라인건설
연결된 재무제표를 찾을 수 없음
대륭산업
연결된 재무제표를 찾을 수 없음
한국종합철관
연결된 재무제표를 찾을 수 없음


In [13]:
'''
시작년도를 5년전으로 해도 재무제표가 없는 기업이 44업체
재무제표가 있는 기업이여도 포괄손익계산서 없는 업체가 총 5개 업체
현금 흐름도가 없는 업체는 총 1개 업체
제무상태표는 모두 있는 상태.
'''
len(thrd_none_financial), len(thrd_none_cis), len(thrd_none_bs), len(thrd_none_cf)

(43, 6, 0, 1)

In [14]:
# 연결된 재무제표가 없는 기업들
for nf in thrd_none_financial:
    print(nf)

에이치앤아이
티에스엠텍
우양에이치씨
제이콤
인네트
신세계톰보이
에스피코프
쌈지
유리이에스
아이비진
우영
알루코
케이티씨텔레콤
아이텍스필
현주컴퓨터
넥스텔
지에스엔텍
현대멀티캡
중앙제지
테크메이트
텔슨전자
금강화섬
호성
대호
모닷텔
코리아링크
시스컴
정산애강
소프트윈
심스밸리
코닉스
유니씨앤티
아이씨켐
디지텔
삼한콘트롤스
정일이엔씨
삼산
교하산업
아진산업
주화산업
라인건설
대륭산업
한국종합철관


In [15]:
# 연결된 재무제표가있으나 포괄손익 계산서가 없는 업체
for nf in thrd_none_cis:
    print(nf)

금오하이텍
카라반케이디이
화림모드
서통
계몽사
피어리스


In [16]:
# 연결된 재무제표가있으나 제무상태표가 없는 업체
for nf in thrd_none_bs:
    print(nf)

In [17]:
# 연결된 재무제표가있으나 현금 흐름도가 없는 업체
for nf in thrd_none_cf:
    print(nf)

화림모드


In [21]:
kosdaq = pd.read_csv('data/kosdaq.csv')

In [22]:
kosdaq

,name
0,위닉스
1,태광
2,한국정밀기계
3,상상인인더스트리
4,바른테크놀로지
...,...
79,세운메디칼
80,에이프런티어
81,동국산업
82,아이원스


In [23]:
have_2019_ko = []
have_2018_ko = []
have_2017_ko = []
none_have_ko = []

In [25]:
crp_list = dart.get_corp_list()

In [ ]:
for ko_list in kosdaq['name']:
    basic_info = crp_list.find_by_corp_name(ko_list, exactly=True)[0]
    try:
        print('2019: ' + ko_list)
        fs = basic_info.extract_fs(bgn_de=st2019)
        have_2019_ko.append(ko_list)
    except:
        try:
            print('2018: ' + ko_list)
            fs = basic_info.extract_fs(bgn_de=st2018)
            have_2018_ko.append(ko_list)
        except:
            try:
                print('2017: ' + ko_list)
                fs = basic_info.extract_fs(bgn_de=st2017)
                have_2017_ko.append(ko_list)

            except:
                print('None: ' + ko_list)
                none_have_ko.append(ko_list)

2019: 위닉스



2019: 태광
2018: 태광
2017: 태광
None: 태광
2019: 한국정밀기계


Output()

Output()

2018: 한국정밀기계


...

Output()

Output()

None: 우림기계
2019: DMS


Output()

Output()